# Advanced Medical Text Extraction 2

Here we will work to use mutiple adnvanced NLP models to validate and test how much advanced medical data such as the level and status of a particular ailment can be extracted. We wil start with examinations around PCa based metastatis - and see if we can identify -

* pca meta confirmed
* pca no meta
* pca unkown
* other metastais info

The Goal here being to unearth the best model or combination of models to be used for further NLP training around that target data down the line 

<br/>

Reaserch Links -
https://colab.research.google.com/drive/1uqc5urkMetu_6J1vU7bTYC6t4JsQ5Arl#scrollTo=ncrDHyf7vPp_

https://colab.research.google.com/drive/1pW0hOXz6OTB7bxRTuaM8rnWMM2mGTOFh


https://colab.research.google.com/drive/1z42H6J_7qGEnNdBodGsZ1G3-xtnoGRxz

In [ ]:
Keyword_List = ['bone scan',' LAD ','Lymphadenopathy','mets', 'metastasis', 'metastatic', 'prostate cancer', 'CaP', 'PCa',
'mCRPC','mHRPC', 'xofigo', 'Radium 223', 'Zytiga', 'Abiraterone', 'Jevtana', 'Cabazitaxel', 'Pluvicto'
'Lutetium Lu 177', 'Vipivotide', 'Tetraxetan', 'Provenge', 'Sipuleucel-T']

PCa_keys = ['prostate cancer', 'CaP', 'PCa']
Mets_keys = ['mets', 'metastasis', 'metastatic']

Keyword_List

## BERT NER 
First lets see what we are able to extract directly with BERT NER -

bert-base-NER is a fine-tuned BERT model that is ready to use for Named Entity Recognition and achieves state-of-the-art performance for the NER task. It has been trained to recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).Jul 8, 2022

### Gather installs and Imports needed 

In [ ]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 32.3 MB/s 
     |████████████████████████████████| 120 kB 56.8 MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

### Set up the example text

In [ ]:
Text = '''patient Marie Bucher was broght to the hospital at 12:00pm last week. She shows Incomplete initial
 resection high-risk recurrent of NMIBC, we detected High-Grade tumors at grade 3 moderate recommendation for treatment'''

### Pull in the model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

### Use the model to extract NER - 

In [ ]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(Text)
print(' Full results')
print(ner_results)
print()
print('Results Break down')

for x in ner_results:
  print(x)

 Full results
[{'entity': 'B-PER', 'score': 0.9995789, 'index': 2, 'word': 'Marie', 'start': 8, 'end': 13}, {'entity': 'I-PER', 'score': 0.9997427, 'index': 3, 'word': 'B', 'start': 14, 'end': 15}, {'entity': 'I-PER', 'score': 0.997968, 'index': 4, 'word': '##uche', 'start': 15, 'end': 19}, {'entity': 'I-PER', 'score': 0.9964156, 'index': 5, 'word': '##r', 'start': 19, 'end': 20}, {'entity': 'B-ORG', 'score': 0.9488242, 'index': 36, 'word': 'N', 'start': 133, 'end': 134}]

Results Break down
{'entity': 'B-PER', 'score': 0.9995789, 'index': 2, 'word': 'Marie', 'start': 8, 'end': 13}
{'entity': 'I-PER', 'score': 0.9997427, 'index': 3, 'word': 'B', 'start': 14, 'end': 15}
{'entity': 'I-PER', 'score': 0.997968, 'index': 4, 'word': '##uche', 'start': 15, 'end': 19}
{'entity': 'I-PER', 'score': 0.9964156, 'index': 5, 'word': '##r', 'start': 19, 'end': 20}
{'entity': 'B-ORG', 'score': 0.9488242, 'index': 36, 'word': 'N', 'start': 133, 'end': 134}


As we see here basic NER is more conserned with names than anything else and can't get much info for use around the medical Targets Themselves we will have to Dig Deeper -

#### BERT NER -Final Model Score - Non Viable For Use

## Using Key Bert
Next lets try a more advanced for of text extraction around any and all Key targets being disscussed if we can find these in our extraction we can pair it with regex to futher fillter results and vastly reduce the text we need to search for our targets 



KeyBERT is a minimal and easy-to-use keyword extraction technique that leverages BERT embeddings to create keywords and keyphrases that are most similar to a document.

Although there are already many methods available for keyword generation (e.g., Rake, YAKE!, TF-IDF, etc.) I wanted to create a very basic, but powerful method for extracting keywords and keyphrases. This is where KeyBERT comes in! Which uses BERT-embeddings and simple cosine similarity to find the sub-phrases in a document that are the most similar to the document itself.



In [ ]:
!pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.0 MB/s 
     |████████████████████████████████| 235 kB 30.0 MB/s 
     |████████████████████████████████| 51 kB 4.7 MB/s 
     |████████████████████████████████| 1.3 MB 57.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=0d73a42ed2b34567e5340faf34cb9e714cdadbb4205d19c47073fc1d404f3eae
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


Now that we have Key Bert Set Up Lets Use it to Extract Targets from our Text

In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT()

keys = kw_model.extract_keywords(Text, keyphrase_ngram_range=(3, 3), top_n = 10, stop_words='english', use_mmr=True, diversity=0.5)
keys 

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

[('patient marie bucher', 0.6425),
 ('nmibc detected high', 0.4358),
 ('bucher broght hospital', 0.42),
 ('grade tumors grade', 0.3986),
 ('risk recurrent nmibc', 0.3519),
 ('resection high risk', 0.3492),
 ('hospital 12 00pm', 0.248),
 ('moderate recommendation treatment', 0.2012),
 ('week shows incomplete', 0.1456),
 ('shows incomplete initial', 0.1298)]

Now This is much better we are seeing information pulled around Grade and level of Risk as well as the specific condition NMIBC but we have some issues.

* 1 we have a lot going on around more than just NMIBC grade
* 2 info on it seperated and we dont get the specifc call around grade 3 
* 3 this simply classfys these sections of text as impotant in the sentence it dose not give us any other form of clasification 

<br/>
Lets See if we can do better with a more medicaly targeted clasification based search


#### Key BERT -Final Model Score - Limited Viablity For Use

## Scispacy Medical Detection
Now we will use a model leveraging spacy based NLP central models but pre trained on ons of medical text to help further identify the targets of medical interest inside the actualcentral text


scispaCy is an open-source software library for advanced Natural Language Processing, written in the programming languages Python and Cython. The library is published under the MIT license and currently offers statistical neural network models for processing biomedical, scientific or clinical text.

In [ ]:
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

Now import in the tools for Bio based NER that we need -

In [ ]:
import scispacy
import spacy
nlp = spacy.load("en_ner_bc5cdr_md") 

Now lets see what we can extract using it on our central targets 


In [ ]:
doc = nlp(Text)
print("TEXT", "START", "END", "ENTITY TYPE")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

TEXT START END ENTITY TYPE
NMIBC 133 138 DISEASE
High-Grade tumors 152 169 DISEASE


Now thats more like it not only was the NBIM condition detected as primeary info we also got the whole Hig Grade state identified as a issuea as well it missed the grade three call but thats all it missed from what we are looking for and that is impressive With aaditonal traning this model could be exactly what we need to gather the data we seek -

#### Scispacy -Final Model Score - Decent Viability For Use

## Stanza Medical Detection 
Next We Will try to a test with the model framework Stanza. Stanza is a Python NLP toolkit that supports 60+ human languages. It is built with highly accurate neural network components that enable efficient training and evaluation with your own annotated data, and offers pretrained models on 100 treebanks. Additionally, Stanza provides a stable, officially maintained Python interface to Java Stanford CoreNLP Toolkit.

Note that Stanza only supports Python 3.6 and above. Installing and importing Stanza are as simple as running the following commands:

In [ ]:
!pip install Stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 4.9 MB/s 
     |████████████████████████████████| 216 kB 45.5 MB/s 
  Created wheel for emoji: filename=emoji-2.1.0-py3-none-any.whl size=212392 sha256=a7d89cabafe6b4d95d2e1613f825025d66bc132e12918ee5a380c014438f5619
  Stored in directory: /root/.cache/pip/wheels/77/75/99/51c2a119f4cfd3af7b49cc57e4f737bed7e40b348a85d82804
Successfully built emoji


Next download the english model

In [ ]:
# Import the package
import stanza

# Download an English model into the default directory
print("Downloading English model...")
stanza.download('en')

INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


## Processing Text
Constructing Pipeline To process a piece of text, you'll need to first construct a Pipeline with different Processor units. The pipeline is language-specific, so again you'll need to first specify the language (see examples).

In [ ]:
# Build an English pipeline, with all processors by default
print("Building an English pipeline...")
en_nlp = stanza.Pipeline('en',verbose=True)

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


Building an English pipeline...


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: constituency
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


## Annotating Text
After a pipeline is successfully constructed, you can get annotations of a piece of text simply by passing the string into the pipeline object. The pipeline will return a Document object, which can be used to access detailed annotations from. For example:

In [ ]:
# Processing English text
en_doc = en_nlp(Text)
print(type(en_doc))

<class 'stanza.models.common.doc.Document'>


We can now have the model perform a full prediction on the structure and anotation of the text-

In [ ]:
for i, sent in enumerate(en_doc.sentences):
    print("[Sentence {}]".format(i+1))

    for word in sent.words:
        print("{:12s}\t{:12s}\t{:6s}\t{:d}\t{:12s}".format(\
              word.text, word.lemma, word.pos, word.head, word.deprel))
        
    print("")

[Sentence 1]
patient     	patient     	NOUN  	5	nsubj       
Marie       	Marie       	PROPN 	5	nsubj       
Bucher      	Bucher      	PROPN 	2	flat        
was         	be          	AUX   	5	cop         
broght      	broght      	ADJ   	0	root        
to          	to          	ADP   	8	case        
the         	the         	DET   	8	det         
hospital    	hospital    	NOUN  	5	obl         
at          	at          	ADP   	11	case        
12:00       	12:00       	NUM   	5	obl         
pm          	pm          	NOUN  	5	obl         
last        	last        	ADJ   	13	amod        
week        	week        	NOUN  	11	nmod:tmod   
.           	.           	PUNCT 	5	punct       

[Sentence 2]
She         	she         	PRON  	2	nsubj       
shows       	show        	VERB  	0	root        
Incomplete  	incomplete  	ADJ   	9	amod        
initial     	initial     	ADJ   	9	amod        
resection   	resection   	NOUN  	9	compound    
high        	high        	ADJ   	8	amod        
-         

This can help us see if the things we are looking for can be indentifited by their sent struct type directly here it looks like we may have to many targets for that so lets move on to more cognative based extraction 

## Now lets perform Basic NER
Running the NERProcessor simply requires the TokenizeProcessor. After the pipeline is run, the Document will contain a list of Sentences, and the Sentences will contain lists of Tokens. Named entities can be accessed through Document or Sentence’s properties entities or ents. Alternatively, token-level NER tags can be accessed via the ner fields of Token.

Accessing Named Entities for Sentence and Document Here is an example of performing named entity recognition for a piece of text and accessing the named entities in the entire document:

In [ ]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')

doc = nlp(Text)
print()
print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in doc.ents], sep='\n')


INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!



entity: Marie Bucher	type: PERSON
entity: 12:00pm	type: TIME
entity: last week	type: DATE
entity: NMIBC	type: ORG
entity: 3	type: CARDINAL


Interesting and Much Better than BERT we got names times and numbers as well as a tracking of NMIBC although it thinks its a ORG this is very good for pulling out key data unfortenetly it dose seem to be unable to detect much around grade besides the number 3 so for this task the viabiity is low but for getting more data around the grade it might be perfect.

### Stansa NER -Final Model Score - Low Viability For Use

## Stansa Medical Text Extraction
Now lets use Stanzas medical libs to define our text further

In [ ]:
stanza.download('en', package='mimic', processors={'ner': 'i2b2'})
nlp = stanza.Pipeline('en', package='mimic', processors={'ner': 'i2b2'})

doc = nlp(Text)

# print out the entities
print()
for ent in doc.entities:
    print(f'{ent.text}\t{ent.type}') 

INFO:stanza:Downloading these customized packages for language: en (English)...
| Processor       | Package |
-----------------------------
| tokenize        | mimic   |
| pos             | mimic   |
| lemma           | mimic   |
| depparse        | mimic   |
| ner             | i2b2    |
| backward_charlm | mimic   |
| pretrain        | mimic   |
| forward_charlm  | mimic   |

INFO:stanza:File exists: /root/stanza_resources/en/tokenize/mimic.pt
INFO:stanza:File exists: /root/stanza_resources/en/pos/mimic.pt
INFO:stanza:File exists: /root/stanza_resources/en/lemma/mimic.pt
INFO:stanza:File exists: /root/stanza_resources/en/depparse/mimic.pt
INFO:stanza:File exists: /root/stanza_resources/en/ner/i2b2.pt
INFO:stanza:File exists: /root/stanza_resources/en/backward_charlm/mimic.pt
INFO:stanza:File exists: /root/stanza_resources/en/pretrain/mimic.pt
INFO:stanza:File exists: /root/stanza_resources/en/forward_charlm/mimic.pt
INFO:stanza:Finished downloading models and saved to /root/stanza_re

INFO:stanza:Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | mimic   |
| pos       | mimic   |
| lemma     | mimic   |
| depparse  | mimic   |
| ner       | i2b2    |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!



Incomplete initial
 resection	TREATMENT
NMIBC	PROBLEM
High-Grade tumors	PROBLEM
treatment	TREATMENT


Very nice even better than sci spacy not only did we identify NMIBC and High Grade Tumors as the problem but we also were able to ID some of the text around treatment this is huge we colud still be a bit more focused on grade but that can come with a little model tuning around our labeled examples and further rules based extraction. As a starting point for the central NLP model this shows some real promise.

### Stanza Medical NER -Final Model Score - Strong Viability For Use

# BioBert NER
now lets try using Bio Bert, BioBERT is a contextualized language representation model, based on BERT, a pre-trained model which is trained on different combinations of general & biomedical domain corpora. One major problem with domain problems is that you have domain texts which are only understood by domain experts

In [ ]:
!pip install -q transformers
!pip install -q simpletransformers

     |████████████████████████████████| 4.9 MB 8.8 MB/s 
     |████████████████████████████████| 6.6 MB 49.8 MB/s 
     |████████████████████████████████| 163 kB 72.2 MB/s 
     |████████████████████████████████| 250 kB 35.4 MB/s 
     |████████████████████████████████| 1.3 MB 57.3 MB/s 
     |████████████████████████████████| 1.8 MB 63.1 MB/s 
     |████████████████████████████████| 431 kB 71.3 MB/s 
     |████████████████████████████████| 9.2 MB 58.2 MB/s 
     |████████████████████████████████| 43 kB 505 kB/s 
     |████████████████████████████████| 181 kB 62.9 MB/s 
     |████████████████████████████████| 162 kB 56.3 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 158 kB 67.0 MB/s 
     |████████████████████████████████| 157 kB 73.6 MB/s 
     |████████████████████████████████| 157 kB 78.2 MB/s 
     |████████████████████████████████| 157 kB 70.5 MB/s 
     |████████████████████████████████| 157 kB 76.5 MB/s 
     |█████████████

Now lets import the Bio Bert model we need as well as anything else we would like to have to help with data trasformation or evaluation.

In [ ]:
from simpletransformers.ner import NERModel
from transformers import AutoTokenizer
import pandas as pd
import logging

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

# We use the bio BERT pre-trained model.
#model = NERModel('bert', 'dmis-lab/biobert-v1.1',use_cuda=False ,labels=custom_labels, args=train_args)
model = NERModel('bert', 'dmis-lab/biobert-v1.1', use_cuda=True)


Downloading:   0%|          | 0.00/462 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

### Using the Model (Running Inference)
Running the model to do some predictions/inference is as simple as calling model.predict(samples). **bold text**

In [ ]:
samples = [Text]

predictions, _ = model.predict(samples)
for idx, sample in enumerate(samples):
  print('{}: '.format(idx))
  for word in predictions[idx]:
    print('{}'.format(word))

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

0: 
{'patient': 'B-PER'}
{'Marie': 'B-MISC'}
{'Bucher': 'B-PER'}
{'was': 'B-PER'}
{'broght': 'B-PER'}
{'to': 'I-LOC'}
{'the': 'I-PER'}
{'hospital': 'B-PER'}
{'at': 'I-PER'}
{'12:00pm': 'B-MISC'}
{'last': 'B-PER'}
{'week.': 'B-PER'}
{'She': 'B-PER'}
{'shows': 'B-PER'}
{'Incomplete': 'O'}
{'initial': 'B-PER'}
{'resection': 'B-PER'}
{'high-risk': 'B-PER'}
{'recurrent': 'B-PER'}
{'of': 'B-PER'}
{'NMIBC,': 'B-PER'}
{'we': 'B-PER'}
{'detected': 'I-PER'}
{'High-Grade': 'I-LOC'}
{'tumors': 'B-PER'}
{'at': 'I-LOC'}
{'grade': 'B-PER'}
{'3': 'I-LOC'}
{'moderate': 'B-PER'}
{'recommendation': 'B-PER'}
{'for': 'B-PER'}
{'treatment': 'B-PER'}


As we can see the model can id text but in its own format and without much specific significance given on onset to our targets a lot of training would be needed to get this model to work for our needs but it dose run well with GPU support so there are advanateges to using it if we have the time and labeled examples to do so. 

### Bio Bert NER -Final Model Score - Low Viability For Use - Without Extensive Training 

## BERT Question Answer
BERT is a Bidirectional Encoder Representations from Transformers. It is one of the most popular and widely used NLP models. BERT models can consider the full context of a word by looking at the words that come before and after it, which is particularly useful for understanding the intent behind the query asked. Because of its bidirectionality, it has a deeper sense of language context and flow and hence, is used in a lot of NLP tasks nowadays. More details about BERT in the article along with the code.

Transformers library has a lot of different BERT models. It is easy to find a task-specific model from this library and do our task. So, let’s get started but let’s first look at our dataset.

### Asking questions direct
CoQA is a Conversational Question Answering dataset released by Stanford NLP in 2019. It is a large-scale dataset for building Conversational Question Answering Systems. This dataset aims to measure the ability of machines to understand a text passage and answer a series of interconnected questions that appear in a conversation. The unique feature about this dataset is that each conversation is collected by pairing two crowd workers to chat about a passage in the form of questions and answers and hence, the questions are conversational. To understand the format of the JSON data, please refer to this link. We will be using the story, question, and answer from the JSON dataset to form our data frame.

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 30.7 MB/s 
     |████████████████████████████████| 163 kB 63.2 MB/s 
     |████████████████████████████████| 6.6 MB 61.1 MB/s 


import needed libs


In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

But for question answering tasks, we can even use the already trained model and get decent results even when our text is from a completely different domain. To get decent results, we are using a BERT model which is fine-tuned on the SQuAD benchmark. For our task, we will use the BertForQuestionAnswering class from the transformers library.

In [ ]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

From here lets build a function that can take in text tokenize it and send it to the model 

In [ ]:
def question_answer(question, text):
    
    #tokenize question and text as a pair
    input_ids = tokenizer.encode(question, text)
    
    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    #segment IDs
    #first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)
    #number of tokens in segment A (question)
    num_seg_a = sep_idx+1
    #number of tokens in segment B (text)
    num_seg_b = len(input_ids) - num_seg_a
    
    #list of 0s and 1s for segment embeddings
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    
    #model output using input_ids and segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    
    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
                
    if answer.startswith("[CLS]"):
        answer = "Unable to find the answer to your question."
    
    print("\nPredicted answer:\n{}".format(answer.capitalize()))

Now lets put it all togther

In [ ]:
Text = '''patient Marie Bucher was broght to the hospital at 12:00pm last week. She shows Incomplete initial
 resection high-risk recurrent of NMIBC, we detected High-Grade tumors at grade 3 moderate recommendation for treatment'''

In [ ]:
question = 'tell me about grade'

question_answer(Text, question)


Predicted answer:
She shows incomplete initial resection high - risk recurrent of nmibc , we detected high - grade tumors


In [ ]:
question = 'tell me everything about grade'

question_answer(Text, question)


Predicted answer:
We detected high - grade tumors at grade 3


## And Thats The Game Folks
As you can see with a little creative text promting around our exact needs we get all the data on NMIBC grade from the text clearly identified from our sample and use it to create full structured data around our cilnical reaserch tagets. 

This is huge and even though Bert Q and A is expenisive we can create a hybrid model with regex that only uses it to get the exact info on grade when any info on grade is detected intially thus saveing massive ammounts of processing power and increasing our speeds ten fold 


## Bert Q and A -Final Model Score - Massive Viability For Use 